In this code, the estimate_pi function estimates the value of Pi by generating n random points in the unit square and counting how many fall within the unit circle. This is done num_estimates times, and the results are averaged to get a more accurate estimate. The computation is distributed across your Dask cluster using the delayed function.

This is a probabilistic method and the accuracy of the estimate increases with the number of points n and the number of estimates num_estimates. You can adjust these parameters to control the computational load.


In [1]:
from dask.distributed import Client
from dask import delayed
import random
import time

def estimate_pi(n):
    """Estimate Pi using a Monte Carlo method."""
    num_points_circle = 0
    num_points_total = 0

    for _ in range(int(n)):
        x = random.uniform(0, 1)
        y = random.uniform(0, 1)
        distance = x**2 + y**2

        if distance <= 1:
            num_points_circle += 1
        num_points_total += 1

    return 4 * num_points_circle / num_points_total

if __name__ == "__main__":
    client = Client('tcp://192.168.1.200:8786')

    # Number of points for each estimate
    n = 10**6

    # Number of estimates
    num_estimates = 100

    results = []
    start_time = time.time()
    for _ in range(num_estimates):
        result = delayed(estimate_pi)(n)
        results.append(result)

    estimates = delayed(lambda x: sum(x) / num_estimates)(results)
    print("Estimated value of Pi: ", estimates.compute())
    end_time = time.time()
    
    print("Time to run: ", end_time - start_time, "seconds")


/home/jamessmithies/Dropbox/Technical/venvs/dask/venv/lib/python3.10/site-packages/distributed/client.py:1391: VersionMismatchWarning: Mismatched versions found

+-------------+-----------------+-----------------+--------------------------------------------------------+
| Package     | Client          | Scheduler       | Workers                                                |
+-------------+-----------------+-----------------+--------------------------------------------------------+
| cloudpickle | 3.0.0           | 3.0.0           | {'2.2.1', '3.0.0'}                                     |
| msgpack     | 1.0.7           | 1.0.7           | {'1.0.5', '1.0.7'}                                     |
| numpy       | 1.26.1          | 1.26.1          | {'1.25.2', '1.26.1', None}                             |
| pandas      | 2.1.1           | 2.1.1           | {'2.0.3', '2.1.1', None}                               |
| python      | 3.10.12.final.0 | 3.10.12.final.0 | {'3.9.2.final.0', '3.10

TypeError: 'range_iterator' object is not callable